In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as sm
from matplotlib import pyplot as plt

In [ ]:
sns.set_context('poster')
sns.set_style('white')
plt.rcParams['text.usetex']=True

In [ ]:
opd = pd.read_hdf("/data/research/results/simdata/xrays/optical_depth.hdf5",'optical_depth')

In [ ]:
opd = opd[opd.view != 'native']

In [ ]:
opd['log n'] = np.log10(opd['n'])
opd['log N'] = np.log10(opd['N'])
opd['log tau'] = np.log10(opd['tau'])
opd['orientation'] = 'random'
opd.orientation[opd.view == 'face (xy)'] = 'polar'
opd.orientation[opd.view == 'xz'] = 'equator'
opd.orientation[opd.view == 'yz'] = 'equator'

In [ ]:
opd.head()

In [ ]:
g = sns.FacetGrid(opd, col='orientation', hue='snapshot', size=5)
g.map(plt.loglog, 'z', 'N')
g.set(xlim=(1, 9e5), ylim=(1e21, 1e28))
g.set_xlabels("Radius [AU]")
g.set_ylabels('Column Density [cm$^{-2}$]')
g.fig.subplots_adjust(wspace=.02)
#g.fig.savefig('figures/optical_depth/column_density.png', bbox_inches='tight')
g.fig.savefig('figures/optical_depth/column_density.pdf', bbox_inches='tight')

In [ ]:
op_slice = opd[opd.n > 1e4]

In [ ]:
g = sns.lmplot("log n", "log N", hue="orientation", data=op_slice[::100], size=6)
g.set_xlabels('Log n [cm$^{-3}$]')
g.set_ylabels('Log N [cm$^{-2}$]')
#g.fig.savefig('figures/optical_depth/col_density_fit.png', bbox_inches='tight', dpi=100)
g.fig.savefig('figures/optical_depth/col_density_fit.pdf', bbox_inches='tight', rasterized=True)

In [ ]:
op_slice['Eins'] = np.ones(( len(op_slice), ))
op_polar = op_slice[op_slice.orientation == 'polar']
op_equator = op_slice[op_slice.orientation == 'equator']

In [ ]:
yp = op_polar['log N']
xp = op_polar[['log n', 'Eins']]
pmodel = sm.OLS(yp, xp)
presults = pmodel.fit()
presults.summary()

In [ ]:
ye = op_equator['log N']
xe = op_equator[['log n', 'Eins']]
emodel = sm.OLS(ye, xe)
eresults = emodel.fit()
eresults.summary()

In [ ]:
print 'Polar Parameters: \n', presults.params
print 'Polar R2: ', presults.rsquared
print '\nEquatorial Parameters: \n', eresults.params
print 'Equatorial R2: ', eresults.rsquared

# Double-check the fit:

In [ ]:
plt.scatter(op_polar['log n'], op_polar['log N'])
logn = np.linspace(0,13)
logN = 0.532301 * logn + 19.636552 
plt.plot(logn, logN)

#Calculate $\tau_{\rm \small effective}$ from $\tau_{\rm \small pole}$ and $\tau_{\rm \small equator}$.

###$\tau_{\rm \small effective} = \tau_{\rm \small pole} \frac{2 \Omega_{\rm \small pole}}{4\pi} + \tau_{\rm \small equator} \frac{4\pi - 2 \Omega_{\rm \small pole}}{4\pi}$

###$\Omega_{\rm \small pole} = \int_0^{2\pi}{\rm d}\phi \int_0^{\pi/4}{\rm sin}\theta \,{\rm d}\theta  = 2\pi \left( 1 - \frac{1}{\sqrt{2}} \right) \approx 2\pi \times 0.29289 = 1.840282$

#From our fit,
##$\tau_{\rm \small pole} = 10^{0.532301 {\rm log_{10}}(n) + 19.636552}$ and $\tau_{\rm \small equator} = 10^{0.626204 {\rm log_{10}}(n) + 19.573490}$

In [ ]:
def tau_p(n):
    exponent = 0.532301 * np.log10(n) + 19.636552
    return 10**exponent

def tau_eq(n):
    exponent = 0.626204 * np.log10(n) + 19.573490
    return 10**exponent

In [ ]:
Omega_pole = 1.840282
fourpi = 4 * np.pi
a = 2 * Omega_pole / fourpi
b = (fourpi - 2*Omega_pole)/fourpi
def tau_eff(n):
    x = tau_p(n)
    y = tau_eq(n)
    return a*x + b*y

### Let's check the results:

In [ ]:
Omega_pole, 4*np.pi

In [ ]:
a,b

In [ ]:
n = np.logspace(0,12)
plt.loglog(n, tau_p(n), '--')
plt.loglog(n, tau_eq(n), '--')
plt.loglog(n, tau_eff(n))
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('N [cm$^{-2}$]')

In [ ]:
n = np.linspace(1e10,1e12)
plt.plot(n, tau_p(n), '--')
plt.plot(n, tau_eq(n), '--')
plt.plot(n, tau_eff(n))
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('N [cm$^{-2}$]')

In [ ]:
n = np.linspace(1,10)
plt.plot(n, tau_p(n), '--')
plt.plot(n, tau_eq(n), '--')
plt.plot(n, tau_eff(n))
plt.xlabel('n [cm$^{-3}$]')
plt.ylabel('N [cm$^{-2}$]')